## Branch Getters

In [1]:
import os 
os.chdir('..')

In [2]:
from utils.format.schema import check_argument, convert_to_bytes_based_on_schema, convert_from_bytes_based_on_schema
from initialization.notebooks.genesis_branch import test_create_genesis_branch_with_initial_submit
from initialization.notebooks.genesis_branch_rpc import test_rpc_create_genesis_branch_with_initial_submit
import inspection.branch as inspection_branch

IOError: b'IO error: lock /tmp/lakat_test//LOCK: Resource temporarily unavailable'

In [3]:
test_genesis_response = test_create_genesis_branch_with_initial_submit(debug=False)

### GETTERS WITHOUT RPC SERVER CALLS

#### Get Branch Name From Branch Id

In [4]:
kwargs = dict(branch_id=test_genesis_response["decoded_branch_id"])
arguments_valid_schema = check_argument(arg=kwargs, schema=inspection_branch.get_branch_name_from_branch_id_schema)
print("arguments are valid according to schema: ", arguments_valid_schema)
converted_kwargs = convert_to_bytes_based_on_schema(schema=inspection_branch.get_branch_name_from_branch_id_schema, data=kwargs)
print("converted_kwargs: ", converted_kwargs)
response = inspection_branch.get_branch_name_from_branch_id(**converted_kwargs)
decoded_response = convert_from_bytes_based_on_schema(schema=inspection_branch.get_branch_name_from_branch_id_schema["response"], data=response)
print("decoded response: ", decoded_response)

arguments are valid according to schema:  True
converted_kwargs:  {'branch_id': b'\x01Q\x12\x02}\xbf'}
decoded response:  Genesis Branch


#### Get Branch Data from Branch Id

In [9]:
bd_kwargs = dict(branch_id=test_genesis_response["decoded_branch_id"], deserialize_buckets=False)
db_arguments_valid_schema = check_argument(arg=bd_kwargs, schema=inspection_branch.get_branch_data_from_branch_id_schema)
print("arguments are valid according to schema: ", db_arguments_valid_schema)
converted_bd_kwargs = convert_to_bytes_based_on_schema(schema=inspection_branch.get_branch_data_from_branch_id_schema, data=bd_kwargs)
print("converted_kwargs: ", converted_bd_kwargs)
bd_response = inspection_branch.get_branch_data_from_branch_id(**converted_bd_kwargs)
decoded_db_response = convert_from_bytes_based_on_schema(schema=inspection_branch.get_branch_data_from_branch_id_schema["response"], data=bd_response)
decoded_db_response
# bd_response


arguments are valid according to schema:  True
converted_kwargs:  {'branch_id': b'\x01Q\x12\x02\x04z', 'deserialize_buckets': False}


{'id': 'AVESAgR6',
 'ns': 'FL0AAA==',
 'name': 'Genesis Branch',
 'parent_id': '',
 'stable_head': {'parent_submit_id': 'AVESAs9tDQQUvQAA',
  'submit_msg': 'Initial Submit'},
 'config': {'accept_conflicts': False, 'branch_type': 1},
 'sprouts': [],
 'sprout_selection': [],
 'name_resolution': 'AVESAsTgEAQUvQAA',
 'interaction': 'AVESAjSNEQQUvQAA',
 'signature': '',
 'creation_ts': 1705187625,
 'submit_trace': {'new_buckets': ['AVESAtrKAQ==',
   'AVESAm0/AQ==',
   'AVESAsOaAg=='],
  'new_registered_names': [{'name': 'Dummy Article Name',
    'id': 'AVESAsOaAg=='}]}}

### Getters via RPC server call

In [2]:
import requests
import json
from config.rpc_cfg import RPC_PORT

In [3]:
def json_rpc_call(method, params=None):
    url = f"http://localhost:{RPC_PORT}/"
    headers = {'content-type': 'application/json'}
    payload = {
        "method": method,
        "params": params,
        "jsonrpc": "2.0",
        "id": 0,
    }
    encoded_payload = json.dumps(payload).encode('utf-8')
    response = requests.post(url, data=encoded_payload, headers=headers)
    # print(response)#
    return response.json()

In [6]:
rpc_genesis_branch_response = test_rpc_create_genesis_branch_with_initial_submit(debug=False)
rpc_genesis_branch_response

{'branch_head_state_id': b'\x01Q\x12\x02\xf65\n\x04\x1c\xdb\x00\x00',
 'decoded_branch_head_state_id': 'AVESAvY1CgQc2wAA',
 'branch_id': b'\x01Q\x12\x02\xf65',
 'decoded_branch_id': 'AVESAvY1'}

#### Get Branch Name Route

In [4]:
rpc_kwargs = dict(branch_id='AVESCC6JnjrcgEU0') #rpc_genesis_branch_response["decoded_branch_id"])
response = json_rpc_call(method="get_branch_name_from_branch_id", params=rpc_kwargs)
name = response['result']
print("Test 'get_branch_name_from_branch_id' passed with branch name:", name)

Test 'get_branch_name_from_branch_id' passed with branch name: Wikipedia Community


#### Get Branch Data Route

In [11]:
rpc_kwargs = dict(branch_id='AVESCC6JnjrcgEU0', deserialize_buckets=True) #rpc_genesis_branch_response["decoded_branch_id"], deserialize_buckets=True)
response = json_rpc_call(method="get_branch_data_from_branch_id", params=rpc_kwargs)

In [13]:
articles = [bucket 
    for bucket in response["result"]["submit_trace"]["new_buckets"]
    if bucket["schema_id"]==2]

In [15]:
articles[0]["data"]

['AVESCKPh2Z6evBoyAQ==',
 'AVESCEInlEDtsAPYAQ==',
 'AVESCKGnaTaaOsyyAQ==',
 'AVESCMhnGewCGLVhAQ==',
 'AVESCKG+Nts4I8VPAQ==',
 'AVESCLi4FZ6mEVpmAQ==',
 'AVESCPR4KZCRiYYpAQ==',
 'AVESCNUhxs7mtR48AQ==',
 'AVESCE4yz5HogoxuAQ==',
 'AVESCHY2WSyY1qPTAQ==',
 'AVESCPamgE53tRAzAQ==',
 'AVESCBZlOJegzIFMAQ==',
 'AVESCEJ74F/tDHLaAQ==',
 'AVESCE2gTihcHdASAQ==',
 'AVESCCfPgx+zN0tDAQ==',
 'AVESCIEGFG6UambdAQ==',
 'AVESCHwM68bcH1LeAQ==',
 'AVESCHVnQEZfcqQAAQ==',
 'AVESCChdXZaUKoynAQ==',
 'AVESCCkCag6JhFmKAQ==',
 'AVESCBxoHsojdne0AQ==',
 'AVESCHNM4nRHgyGHAQ==',
 'AVESCEe+2vUue67ZAQ==',
 'AVESCBSCuFZKeg6QAQ==',
 'AVESCODhqOGy4oYbAQ==',
 'AVESCCp2ynNvj6W+AQ==',
 'AVESCOt8NZKvaNWZAQ==',
 'AVESCEqadiuBDl9iAQ==',
 'AVESCEpH7TKePUDmAQ==',
 'AVESCNIT78XJPY7EAQ==',
 'AVESCL7LJ6UnHykaAQ==',
 'AVESCCRJf9To4y/NAQ==',
 'AVESCJp4NXJ/+m2QAQ==',
 'AVESCCt78YRN/xslAQ==',
 'AVESCGI0wKkHbOKjAQ==',
 'AVESCFlmYPzbOdiuAQ==',
 'AVESCNwOVEEJPOn2AQ==',
 'AVESCNh29K294/rnAQ==',
 'AVESCCNUTawJgRJ0AQ==',
 'AVESCGGtP9l5fvX7AQ==',


In [16]:
from utils.encode.bytes import decode_base64_str_to_bytes
from utils.encode.hashing import get_namespace_from_lakat_cid
bytes_example = decode_base64_str_to_bytes('AVESCKG+Nts4I8VPAQ==')
get_namespace_from_lakat_cid(bytes_example)

1